In [1]:
import os
import autogen
from autogen import AssistantAgent, UserProxyAgent, ConversableAgent
from IPython import get_ipython

In [2]:
OAI_CONFIG_LIST = {
    'model': 'gpt-4o',
    'api_key': 'sk-'
}

In [3]:
OPENAI_MODEL = "gpt-4o"

config_list = [{
    "model": OPENAI_MODEL,
    "api_key": "sk-"
}]

In [4]:
llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "python",
            "description": "run cell in ipython and return the execution result.",
            "parameters": {
                "type": "object",
                "properties": {
                    "cell": {
                        "type": "string",
                        "description": "Valid Python cell to execute.",
                    }
                },
                "required": ["cell"],
            },
        }
    ],
    "config_list": config_list,
    "timeout": 120,
}


In [5]:
PATH=os.getcwd()
batch_size=5
evaluation_sample_size=20

In [6]:
gpt_config = {
    "cache_seed": 42,
    "temperature": 0,
    "config_list": config_list,
    "timeout": 600,
}

In [7]:
# Schema Updater Agent
schema_gen = autogen.AssistantAgent(
    name="schema_gen",
    llm_config=llm_config,
    system_message=f"Specializes in reading court order documents iteratively and generating comprehensive schemas with all important information. Reads and updates schema for only {batch_size} documents at a time, updating the schema after each batch. Schemas should be strictly based on the information in the documents. No dummy schema or hallucination allowed. Assign correct type to the keys based on the information in the documents, not your intuition. Ensure correct variable type for the keys. Use bools for yes/no questions, strings for text, and numbers for numerical values.",
)

# Code Generator Agent
code_generator = autogen.AssistantAgent(
    name="code_generator",
    llm_config=llm_config,
    system_message="Generates necessary code for processing documents and managing schemas, returning it in the required format. Also executes the code. Strictly, only use python code generation."
)

# User Interface Agent
user_interface = autogen.UserProxyAgent(
   name="user_interface",
   system_message="Admin interface for user input. Supports user queries and termination signals. Invoke human feedback after each batch of schema generation and evaluation",
   code_execution_config={"work_dir":"coding", "use_docker": False},
   max_consecutive_auto_reply=3,
   llm_config=llm_config,
   human_input_mode="ALWAYS"
)

closeness_checker = autogen.AssistantAgent(
    name="closeness_checker",
    llm_config=llm_config,
    system_message=(
         f"""Regenerates a random sample of {evaluation_sample_size} percent of original documents in a batch by just using the schema generated till then. It then compares the generated regenerated with the original documents. Evaluates regenerated court order documents to ensure all information from the original document is incorporated.
            By this comparison, the keys/questions not incorprated in the schema are identified and reported.
        Provides detailed feedback on the generated schema of the given batch to optimize the completeness of the schemas. It is strict critic of generated schema and flawlessly ensures that not even minute bit of information is lost. """
    )
)

In [8]:
def exec_python(cell):
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log

def exec_sh(script):
    return user_interface.execute_code_blocks([("sh", script)])

In [9]:
schema_gen.register_function(function_map={
        "python": exec_python
})

code_generator.register_function(function_map={
        "python": exec_python
})
user_interface.register_function(function_map={
        "python": exec_python
})

closeness_checker.register_function(function_map={
        "python": exec_python
})

In [10]:
groupchat = autogen.GroupChat(agents=[user_interface, schema_gen,code_generator,closeness_checker], messages=[], max_round=100000)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt_config)

In [11]:
user_interface.initiate_chat(
    manager,
    message=f""" Use Python only. Return the schema in standard JSON format. Load the text court orders from the provided folder path {PATH}. The files are inside the folder. First and foremost, read the text files for first batch. Calculate number of batches before processing first batch.
    Read the court judgments iteratively, updating and improving the schema each time after reading a batch. Strictly perform document regeneration and closeness checking at the end updating schema for each batch by randomly selecting documents to regenerate and compare. Do not skip this step.
    Focus on generating a standardized schema with keys and their datatypes only, not values. Make it clear, I do not want to store any data from these documents but want to study the strcuture of these documents; that is, what are the questions that are needed to be answered to create document like these.
    """
)


user_interface (to chat_manager):

 Use Python only. Return the schema in standard JSON format. Load the text court orders from the provided folder path e:\schemaGenerator. The files are inside the folder. First and foremost, read the text files for first batch. Calculate number of batches before processing first batch.
    Read the court judgments iteratively, updating and improving the schema each time after reading a batch. Strictly perform document regeneration and closeness checking at the end updating schema for each batch by randomly selecting documents to regenerate and compare. Do not skip this step.
    Focus on generating a standardized schema with keys and their datatypes only, not values. Make it clear, I do not want to store any data from these documents but want to study the strcuture of these documents; that is, what are the questions that are needed to be answered to create document like these.
    

---------------------------------------------------------------------

5

code_generator (to chat_manager):

***** Response from calling function "python" *****
5
***************************************************

--------------------------------------------------------------------------------
schema_gen (to chat_manager):

There are 5 batches of documents to process. Let's start with the first batch of 5 documents and generate the initial schema.

I'll read the first 5 documents and create a schema based on their structure.
***** Suggested function Call: python *****
Arguments: 
{"cell":"import os\n\n# Define the folder path\nfolder_path = \"e:/schemaGenerator\"\n\n# List all text files in the folder\ntext_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]\n\n# Read the first batch of 5 documents\nbatch_size = 5\nfirst_batch_files = text_files[:batch_size]\n\n# Read the content of the first batch of files\nfirst_batch_content = []\nfor file_name in first_batch_files:\n    with open(os.path.join(folder_path, file_name), 'r') as file:\n     

['Court Tehsildar-Sadar, Varanasi. Case number-J202414700104931 year-2024 under section-34/35 UP Revenue Code, Mauza-Bhagwanpur Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nSmt. Madhuri Devi vs. Smt. Sita Devi\n\nDecision\nThis suit of transfer is recorded on the registered sale deed number-1133 dated-16/02/2024 received from the registration office Varanasi. Which was instituted on the basis of the transfer report of the Lekhpal. Out of the area of â€‹â€‹0.3020 hectare of Arazi number-529mi, area of â€‹â€‹690 square feet i.e. area-64.12 square meters situated in Mauza-Bhagwanpur Pargana-Dehat Amanat, Tehsil and District-Varanasi has been sold by Smt. Sita Devi to Mrs. Madhuri Devi. As per rules, the suit was registered and a proclamation letter was issued which was served and the file is attached. In the absence of any objection, the suit is undisputed.\nThe buyer has stated in his evidence that even after purchasing the said land, as per the original sale deed, extract Kha

user_interface (to chat_manager):

***** Response from calling function "python" *****
['Court Tehsildar-Sadar, Varanasi. Case number-J202414700104931 year-2024 under section-34/35 UP Revenue Code, Mauza-Bhagwanpur Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nSmt. Madhuri Devi vs. Smt. Sita Devi\n\nDecision\nThis suit of transfer is recorded on the registered sale deed number-1133 dated-16/02/2024 received from the registration office Varanasi. Which was instituted on the basis of the transfer report of the Lekhpal. Out of the area of â€‹â€‹0.3020 hectare of Arazi number-529mi, area of â€‹â€‹690 square feet i.e. area-64.12 square meters situated in Mauza-Bhagwanpur Pargana-Dehat Amanat, Tehsil and District-Varanasi has been sold by Smt. Sita Devi to Mrs. Madhuri Devi. As per rules, the suit was registered and a proclamation letter was issued which was served and the file is attached. In the absence of any objection, the suit is undisputed.\nThe buyer has stated in his eviden

...

['{\n    "court_name": {\n        "type": "string",\n        "examples": []\n    },\n    "case_number": {\n        "type": "string",\n        "examples": []\n    },\n    "year": {\n        "type": "number",\n        "examples": []\n    },\n    "section": {\n        "type": "string",\n        "examples": []\n    },\n    "mauza": {\n        "type": "string",\n        "examples": []\n    },\n    "pargana": {\n        "type": "string",\n        "examples": []\n    },\n    "tehsil": {\n        "type": "string",\n        "examples": []\n    },\n    "district": {\n        "type": "string",\n        "examples": []\n    },\n    "plaintiff": {\n        "type": "string",\n        "examples": []\n    },\n    "defendant": {\n        "type": "string",\n        "examples": []\n    },\n    "decision": {\n        "sale_deed_number": {\n            "type": "string",\n            "examples": []\n        },\n        "sale_deed_date": {\n            "type": "string",\n            "examples": []\n        },

closeness_checker (to chat_manager):

***** Response from calling function "python" *****
['{\n    "court_name": {\n        "type": "string",\n        "examples": []\n    },\n    "case_number": {\n        "type": "string",\n        "examples": []\n    },\n    "year": {\n        "type": "number",\n        "examples": []\n    },\n    "section": {\n        "type": "string",\n        "examples": []\n    },\n    "mauza": {\n        "type": "string",\n        "examples": []\n    },\n    "pargana": {\n        "type": "string",\n        "examples": []\n    },\n    "tehsil": {\n        "type": "string",\n        "examples": []\n    },\n    "district": {\n        "type": "string",\n        "examples": []\n    },\n    "plaintiff": {\n        "type": "string",\n        "examples": []\n    },\n    "defendant": {\n        "type": "string",\n        "examples": []\n    },\n    "decision": {\n        "sale_deed_number": {\n            "type": "string",\n            "examples": []\n        },\n        

['{\n    "court_name": {\n        "type": "string",\n        "examples": []\n    },\n    "case_number": {\n        "type": "string",\n        "examples": []\n    },\n    "year": {\n        "type": "number",\n        "examples": []\n    },\n    "section": {\n        "type": "string",\n        "examples": []\n    },\n    "mauza": {\n        "type": "string",\n        "examples": []\n    },\n    "pargana": {\n        "type": "string",\n        "examples": []\n    },\n    "tehsil": {\n        "type": "string",\n        "examples": []\n    },\n    "district": {\n        "type": "string",\n        "examples": []\n    },\n    "plaintiff": {\n        "type": "string",\n        "examples": []\n    },\n    "defendant": {\n        "type": "string",\n        "examples": []\n    },\n    "decision": {\n        "sale_deed_number": {\n            "type": "string",\n            "examples": []\n        },\n        "sale_deed_date": {\n            "type": "string",\n            "examples": []\n        },

closeness_checker (to chat_manager):

***** Response from calling function "python" *****
['{\n    "court_name": {\n        "type": "string",\n        "examples": []\n    },\n    "case_number": {\n        "type": "string",\n        "examples": []\n    },\n    "year": {\n        "type": "number",\n        "examples": []\n    },\n    "section": {\n        "type": "string",\n        "examples": []\n    },\n    "mauza": {\n        "type": "string",\n        "examples": []\n    },\n    "pargana": {\n        "type": "string",\n        "examples": []\n    },\n    "tehsil": {\n        "type": "string",\n        "examples": []\n    },\n    "district": {\n        "type": "string",\n        "examples": []\n    },\n    "plaintiff": {\n        "type": "string",\n        "examples": []\n    },\n    "defendant": {\n        "type": "string",\n        "examples": []\n    },\n    "decision": {\n        "sale_deed_number": {\n            "type": "string",\n            "examples": []\n        },\n        

KeyboardInterrupt: 